In [84]:
import numpy as np
from tempfile import TemporaryFile
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [85]:
import numpy as np
from tempfile import TemporaryFile
import pandas as pd
import sklearn.linear_model

features = 'potential,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes'
features_list = features.split(',')
num_features = len(features_list)
features_list = features_list * 30

print(len(features_list))
k=0
for bucket_id in range(30):
    idx = bucket_id  * num_features
    for feature in features_list[ bucket_id  * num_features :  (bucket_id + 1 ) * num_features]:

        features_list[idx] = str(bucket_id) + "_" + features_list[idx]
        idx += 1



def train_validate_test_split(df, train_percent=.8, validate_percent=0, seed=100):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.ix[perm[:train_end]]
    validate = df.ix[perm[train_end:validate_end]]
    test = df.ix[perm[validate_end:]]
    return train, validate, test


outfile = 'alldata_buckets_4x3.npz'
npzfile = np.load(outfile)

all_data = npzfile['arr_0']
print(all_data.shape)

train_data, validate_data, test_data = train_validate_test_split(pd.DataFrame(all_data))

train_data = np.array(train_data)
validate_data = np.array(validate_data)
test_data = np.array(test_data)


train_X = train_data[:, :-1]
validate_X = validate_data[:, :-1]
test_X = test_data[:, :-1]
train_Y = train_data[:,-1]
validate_Y = validate_data[:,-1]
test_Y = test_data[:,-1]
print(train_X.shape)
print(validate_X.shape)
print(test_X.shape)

sc = StandardScaler()
train_X = sc.fit_transform(train_X)
test_X = sc.transform(test_X)
epsilon = 1e-12
mean_train = np.mean(train_X, axis = 0)
std_train = np.std(train_X, axis = 0) + epsilon

test_X = (test_X - mean_train)/std_train

kbest = sklearn.linear_model.LogisticRegression(penalty='l1', C= 5)
kbest.fit(train_X, train_Y)
print('training accuracy', kbest.score(train_X, train_Y))
print('test accuracy', kbest.score(test_X, test_Y))
print('all coefficients', kbest.coef_)
best_features = np.argsort(np.abs(kbest.coef_))[::-1].ravel()
#print('15 best features', best_features)
logcoefs = kbest.coef_.ravel()

1080
(15965L, 649L)
(12772L, 648L)
(0L, 648L)
(3193L, 648L)
training accuracy 0.740056373317
test accuracy 0.70122142186
all coefficients [[  4.28237390e-02   3.15689704e-02  -4.62009089e-02  -4.04765988e-02
   -2.60852611e-02   2.50444473e-02  -1.96037155e-02  -1.10826883e-02
    2.39757725e-02   6.97240325e-03  -4.04153164e-02   7.38022419e-03
    8.15512785e-02  -1.23985338e-01   9.10474793e-02  -5.71099123e-02
    6.12324175e-02   8.39491174e-03  -1.71231398e-02   3.05703678e-02
   -3.85830461e-02   5.43591194e-02   9.26807152e-02   5.96070538e-03
   -5.14090000e-02  -1.25955411e-02   1.58811707e-02   7.51796721e-02
   -6.72374820e-02   9.74577213e-02   1.19207890e-01   4.81716850e-02
    4.77519243e-02   7.13612019e-02  -6.50289356e-03   2.30159678e-02
   -3.30947800e-02  -2.92970552e-04  -1.69258488e-02  -5.98533149e-02
    1.67104240e-02  -3.10745735e-02   2.66206423e-02   7.55703004e-03
    2.51794172e-02   3.25194725e-02   2.01017054e-02   5.46689171e-02
   -2.26968855e-02  -1

In [56]:
best_features = np.argsort(np.abs(kbest.coef_))[::-1].ravel()
print(best_features.shape)
for idx in  range(K):
    print (features_list[idx], ",", kbest.coef_.ravel()[best_features[idx]])

(648L,)
0_potential , 0.0
0_attacking_work_rate , 0.0
0_defensive_work_rate , 0.0
0_crossing , 0.0
0_finishing , 0.0
0_heading_accuracy , 0.0
0_short_passing , 0.0
0_volleys , 0.0
0_dribbling , 0.0
0_curve , 0.0
0_free_kick_accuracy , 0.0
0_long_passing , 0.0
0_ball_control , -2.92255076453e-06
0_acceleration , -1.07588372897e-05
0_sprint_speed , -0.00029297055215
0_agility , -0.000434587760372
0_reactions , -0.000488891507631
0_balance , 0.000720464332473
0_shot_power , 0.000789764788532
0_jumping , 0.00106336299316
0_stamina , -0.00117724843689
0_strength , 0.001764924104
0_long_shots , 0.00227432386798
0_aggression , -0.0028961011199
0_interceptions , -0.00296544622841
0_positioning , 0.00311264012421
0_vision , 0.00328018844047
0_penalties , 0.00402059852327
0_marking , 0.00404989921034
0_standing_tackle , 0.00405762716329
0_sliding_tackle , 0.00412408318685
0_gk_diving , 0.0041602986308
0_gk_handling , -0.00416528533485
0_gk_kicking , -0.00423130631844
0_gk_positioning , -0.004263

In [86]:
import sklearn.metrics
Y_pred = kbest.predict(test_X)
print(classification_report(test_Y, Y_pred))

             precision    recall  f1-score   support

       -1.0       0.60      0.56      0.58      1169
        1.0       0.75      0.79      0.77      2024

avg / total       0.70      0.70      0.70      3193



In [87]:
roc_auc_score(test_Y, Y_pred)

0.67037424304412074

In [88]:
from sklearn.metrics import zero_one_loss
zero_one_loss(test_Y, Y_pred)

0.29877857813968056

In [93]:
from sklearn.linear_model import LogisticRegression

C = np.linspace( 1, 2, 10)
test_scores = np.zeros(10)
scores = np.zeros(10)
for (i,c) in enumerate(C):
    log = LogisticRegression(C = c,penalty='l1')
    cross = cross_val_score(log,train_X,train_Y,cv = 5)
    scores[i] = np.mean(cross)
    log2 = LogisticRegression(C = c,penalty = "l1")
    log2.fit(train_X,train_Y)
    test_scores[i] = log2.score(test_X,test_Y)
    print (c, scores[i],test_scores[i])

1.0 0.70028184943 0.704040087692
1.08163265306 0.700438558449 0.704040087692
1.16326530612 0.70059517552 0.703726902599
1.24489795918 0.700986748847 0.704666457877
1.32653061224 0.701221490559 0.703726902599
1.40816326531 0.701378107631 0.704040087692
1.48979591837 0.700908348364 0.704353272784
1.57142857143 0.700673514705 0.703726902599
1.65306122449 0.700438589098 0.704040087692
1.73469387755 0.70036034186 0.703726902599


IndexError: index 10 is out of bounds for axis 0 with size 10

In [ ]:
bestc = C[np.argmax(scores)]
log = LogisticRegression(C = bestc,penalty = "l1")
log.fit(train_X,train_y)